In [1]:
# Aim is to connect the Movie Names from the file "u.item" to the MovieID from the last notebook.
# Use of "sc.broadcast"

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setMaster("local").setAppName("PopularMovies")
sc = SparkContext.getOrCreate(conf = conf)

In [4]:
data1 = sc.textFile("/FileStore/tables/u.data")  #Movie ID and ratings data
for i in data1.collect()[0:9]:
  print(i)

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817

In [5]:
# The MovieNames are in the file "u.item".
# Let's see the u.item file.

data2 = sc.textFile("/FileStore/tables/u.item")

for i in data2.collect()[0:4]:
  print(i)

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0

In [6]:
# We can see that 1st column is MovieID, 2nd column is MovieName and are seperated by "|".
# Creating a function that extracts the required columns and created a dictionary of MovieID and MovieName.

def loadMovieNames():
  movieNames = {}                           #Empty dictionary
  for line in data2.collect():
    fields = line.split("|")                #Splitting on "|"
    movieNames[int(fields[0])] = fields[1]  #Adding MovieID = MovieName in blank dictionary movieNames
    
    return movieNames

In [7]:
nameDict = sc.broadcast(loadMovieNames())  #Saving the dictionary obtained through the function loadMoviesNames as broadcast object

In [8]:
# Repeating what we did in previous notebook i.e. finding the most popular (watched) movie (METHOD 2)

pairs = data1.map(lambda x: (int(x.split()[1]),1))
count = pairs.reduceByKey(lambda x,y: x+y)
sortedMovies  = count.sortBy(lambda x:x[1], ascending = False)

for i in sortedMovies.collect()[0:4]:
  print (i)

(50, 583)
(258, 509)
(100, 508)
(181, 507)

In [9]:
  # Now we have the 1st column as unique "MovieID" and 2nd column as the 2. "count", we can associate MovidID with MovieName from 'nameDict' as follows. 
# MovieName can be obtained as: dictionary.value(key)  i.e. nameDict.value(MovidID)

sortedMoviesWithNames = sortedMovies.map(lambda x : (nameDict.value[x[0]], x[1]))

for i in sortedMoviesWithNames.collect():
    print(i)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3751579582916567> in <module> 
 4 sortedMoviesWithNames = sortedMovies . map ( lambda x : ( nameDict . value [ x [ 0 ] ] , x [ 1 ] ) ) 
 5 
 ----> 6 for i in sortedMoviesWithNames . collect ( ) : 
 7 print ( i ) 

 /databricks/spark/python/pyspark/rdd.py in collect (self) 
 829 # Default path used in OSS Spark / for non-credential passthrough clusters: 
 830 with SCCallSiteSync ( self . context ) as css : 
 --> 831 sock_info = self . ctx . _jvm . PythonRDD . collectAndServe ( self . _jrdd . rdd ( ) ) 
 832 return list ( _load_from_socket ( sock_info , self . _jrdd_deserializer ) ) 
 833 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 303.0 failed 1 times, most recent failure: Lost task 0.0 in stage 303.0 (TID 358, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 480, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 472, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 508, in dump_stream
 vs = list(itertools.islice(iterator, batch))
 File "/databricks/spark/python/pyspark/util.py", line 99, in wrapper
 return f(*args, **kwargs)
 File "<command-3751579582916567>", line 4, in <lambda>
KeyError: 50

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:540)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:676)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:659)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:494)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:59)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:104)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:48)
	at scala.collection.TraversableOnce$class.to(TraversableOnce.scala:310)
	at org.apache.spark.InterruptibleIterator.to(InterruptibleIterator.scala:28)
	at scala.collection.TraversableOnce$class.toBuffer(TraversableOnce.scala:302)
	at org.apache.spark.InterruptibleIterator.toBuffer(InterruptibleIterator.scala:28)
	at scala.collection.TraversableOnce$class.toArray(TraversableOnce.scala:289)
	at org.apache.spark.InterruptibleIterator.toArray(InterruptibleIterator.scala:28)
	at org.apache.spark.rdd.RDD$$anonfun$collect$1$$anonfun$15.apply(RDD.scala:997)
	at org.apache.spark.rdd.RDD$$anonfun$collect$1$$anonfun$15.apply(RDD.scala:997)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:2321)